# **SISTEMAS DE RECOMENDACIÓN**

## **Filtrado Basado en Contenido**


Miembros del Grupo:
- Paula Arias Fernández
- Jorge del Castillo Gómez
- Anny Álvarez Nogales

In [ ]:
# Importing libraries
import pandas as pd
import re
from joblib import Parallel, delayed

# **Pruebas Datos Textuales** 

## Pruebas iniciales

1. TF-IDF:
    - TFIDF + LogisticRegression -  `MAE: 0.65`
    - TFIDF + RandomForestRegression - `MAE: 0.82`
    - TFIDF + xgboost - `MAE: - 0.65`
2. Doc2Vec + LogisticRegression - `MAE: 1.24`

In [ ]:
# -- Tfidf elimina las stopwords
vectorizer = TfidfVectorizer(stop_words='english', max_features=200, ngram_range=(1, 2))

X_train_tfidf = vectorizer.fit_transform(df_train['text'])
X_test_tfidf = vectorizer.transform(df_test['text'])

In [ ]:
y = df_train['stars']

X_train, X_test, y_train, y_test = train_test_split(X_train_tfidf, y, test_size=0.2, random_state=42)

# model = RandomForestRegressor(n_estimators=100, random_state=42)
model = LogisticRegression(max_iter=1000)

model.fit(X_train, y_train)
y_pred = model.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
print(f'Mean Absolute Error (Train Test Split): {mae}')

y_pred_test = model.predict(X_test_tfidf)

**GridSearch** con LogisticRegression

In [ ]:
y = df_train['stars']
X_train, X_test, y_train, y_test = train_test_split(X_train_tfidf, y, test_size=0.2, random_state=42)

param_grid = {
    'C': [0.01, 0.1, 1, 10, 100],     # Regularización
    'solver': ['liblinear', 'saga'],  # Métodos de optimización
    'max_iter': [100, 500, 1000],     # Número máximo de iteraciones
}

model = LogisticRegression(random_state=42)

grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='neg_mean_absolute_error', n_jobs=-1, verbose=1)

grid_search.fit(X_train, y_train)

best_model = grid_search.best_estimator_
best_params = grid_search.best_params_
y_pred_best = best_model.predict(X_test)
mae_best = mean_absolute_error(y_test, y_pred_best)

print(f"Mejores parámetros: {best_params}")
print(f"Mean Absolute Error (mejor modelo): {mae_best}")

# Comparacion entre el mejor modelo y uno predeterminado (sin optimizar)
model_default = LogisticRegression(max_iter=1000, random_state=42)
model_default.fit(X_train, y_train)
y_pred_default = model_default.predict(X_test)
mae_default = mean_absolute_error(y_test, y_pred_default)
print(f"Mean Absolute Error (modelo sin optimizar): {mae_default}")

### Predicciones
y_pred_test = best_model.predict(X_test_tfidf)

submission_df = pd.DataFrame({
    'review_id': df_test['review_id'],  
    'stars': y_pred_test  
})

submission_df.to_csv('prediction_tfidf_logisticReg_gridSearch.csv', index=False)

**Con un Clasificador RandomForest**

In [ ]:
vectorizer = TfidfVectorizer(stop_words='english', max_features=100, ngram_range=(1, 2))

X_train_tfidf = vectorizer.fit_transform(df_train['text'])
X_test_tfidf = vectorizer.transform(df_test['text'])

### Con RandomForest
y = df_train['stars']

X_train, X_test, y_train, y_test = train_test_split(X_train_tfidf, y, test_size=0.2, random_state=42)

model = RandomForestRegressor(n_estimators=10, random_state=42, n_jobs=-1)

model.fit(X_train, y_train)
y_pred = model.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
print(f'Mean Absolute Error (Train Test Split): {mae}')

y_pred_test = model.predict(X_test_tfidf)

submission_df = pd.DataFrame({
    'review_id': df_test['review_id'],
    'stars': y_pred_test
})

submission_df.to_csv('prediction_tfidf_randomForest.csv', index=False)

**Con un clasificador xgboost**

In [ ]:
import xgboost
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report

y = df_train['stars'].astype(int) - 1

model = XGBClassifier(n_estimators=100, use_label_encoder=False, eval_metric='mlogloss', random_state=42, n_jobs=-1)
model.fit(X_train_tfidf, y)

y_pred_test = model.predict(X_test_tfidf) + 1

submission_df = pd.DataFrame({
    'review_id': df_test['review_id'],
    'stars': y_pred_test
})

submission_df.to_csv('prediction_tfidf_xgboost.csv', index=False)

### **Doc2Vec**
Enlace: https://spotintelligence.com/2023/09/06/doc2vec/

In [ ]:
######################
#### PREPROCESADO ####
######################
print("--- Inicio Preprocesado...")

# Tokenizer
def preprocess_text_parallel(text):
    return re.findall(r'\b[a-zA-Z]+\b', text.lower())

df_train['tokens'] = Parallel(n_jobs=-1)(delayed(preprocess_text_parallel)(text) for text in df_train['text'])
df_test['tokens'] = Parallel(n_jobs=-1)(delayed(preprocess_text_parallel)(text) for text in df_test['text'])

# TaggedDocument
tagged_train = [TaggedDocument(words=tokens, tags=[str(i)]) for i, tokens in enumerate(df_train['tokens'])]

########################
#### MODELO DOC2VEC ####
########################

# Initialize the Doc2Vec model
model = Doc2Vec(vector_size=50,   # Dimensionality of the document vectors
                window=2,         # Maximum distance between the current and predicted word within a sentence
                min_count=1,      # Ignores all words with total frequency lower than this
                workers=-1,       # Number of CPU cores to use for training
                epochs=2)         # Number of training epochs

model.build_vocab(tagged_train)
model.train(tagged_train, total_examples=len(tagged_train), epochs=model.epochs)

# Inferir vectores
df_train['vector'] = df_train['tokens'].apply(lambda x: model.infer_vector(x))
df_test['vector'] = df_test['tokens'].apply(lambda x: model.infer_vector(x))


X_test = list(df_test['vector'])

#############################
#### MODELO CLASIFICADOR ####
#############################

X_list_train = list(df_train['vector'])
y_train = df_train['stars']


classifier = LogisticRegression(max_iter=1000)
classifier.fit(X_list_train, y_train)

predicted_stars = classifier.predict(X_test)

submission_df = pd.DataFrame({
    'review_id': df_test['review_id'],
    'stars': predicted_stars
})
submission_df.to_csv('prediction_doc2vec_logreg.csv', index=False)

## Word2Vec Embedding Model



In [ ]:
#DATOS
train_reviews=pd.read_csv('train_reviews.csv', sep=',')
test_reviews = pd.read_csv('test_reviews.csv')
negocios_df=pd.read_csv('negocios.csv')

df = train_reviews.merge(negocios_df, on='business_id',how='inner')
df

In [ ]:
#PREPROCESS DATA 

df['text'] = df['text'].astype(str) + " " + df['categories'].astype(str)

def preprocess_text_parallel(text):
    return re.findall(r'\b[a-zA-Z]+\b', text.lower())

train_reviews['tokens'] = Parallel(n_jobs=-1)(delayed(preprocess_text_parallel)(text) for text in df['text'])
test_reviews['tokens'] = Parallel(n_jobs=-1)(delayed(preprocess_text_parallel)(text) for text in test_reviews['text'])


In [ ]:
#TEXT MODEL WORD2VEC
#se entrena una vez y se guarda el modelo

import gensim
from gensim.models import Word2Vec
model = Word2Vec(
    sentences=train_reviews['tokens'],
    vector_size=50,  
    window=5,
    min_count=5,  
    workers=4,  
    epochs=7 
)


#model.save("word2vec_model.model")
import gensim
from gensim.models import Word2Vec
model = Word2Vec.load("word2vec_model.model")

In [ ]:
#APPLY TEXT EMBEDDING MODEL
def get_review_vector(tokens, model):
    vectors = [model.wv[word] for word in tokens if word in model.wv]
    if len(vectors) == 0:
        return [0] * model.vector_size  
    return sum(vectors) / len(vectors)

train_reviews['vector'] = train_reviews['tokens'].apply(lambda x: get_review_vector(x, model))

X_train = list(train_reviews['vector'])
y_train = train_reviews['stars']

In [ ]:
#CLASSIFIER LOGISTIC REGRESSION

from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(max_iter=2500,C= 14.867708330182724,solver='newton-cg',penalty='l2')
classifier.fit(X_train, y_train)

test_reviews['vector'] = test_reviews['tokens'].apply(lambda x: get_review_vector(x, model))

X_test = list(test_reviews['vector'])
predicted_stars = classifier.predict(X_test)

submission_df = pd.DataFrame({
    'review_id': test_reviews['review_id'],
    'stars': predicted_stars
})

submission_df

In [ ]:
#CLASSIFIER RANDOMFOREST

from sklearn.ensemble import RandomForestClassifier

classifier = RandomForestClassifier(n_estimators=100, random_state=42)
classifier.fit(X_train, y_train)

test_reviews['vector'] = test_reviews['tokens'].apply(lambda x: get_review_vector(x, model))

X_test = list(test_reviews['vector'])
predicted_stars = classifier.predict(X_test)

submission_df = pd.DataFrame({
    'review_id': test_reviews['review_id'],
    'stars': predicted_stars
})


submission_df

In [ ]:
#XGBOOST CLASSIFIER

from xgboost import XGBClassifier
y_train_adj = y_train - 1

classifier = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42)
classifier.fit(X_train, y_train_adj)

X_test = list(test_reviews['vector'])
predicted_stars = classifier.predict(X_test) + 1  


submission_df = pd.DataFrame({
    'review_id': test_reviews['review_id'],
    'stars': predicted_stars
})

In [ ]:
#OPTUNA WITH LOGISTIC REGRESSION
import optuna
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

# Optimización para el mejor modelo de regresión logística

def objective(trial):
    C = trial.suggest_loguniform('C', 1e-5, 100)  
    solver = trial.suggest_categorical('solver', ['liblinear', 'lbfgs', 'newton-cg', 'saga'])
    max_iter = trial.suggest_int('max_iter', 1000, 5000, step=500)
    penalty = trial.suggest_categorical('penalty', ['l2'])

    classifier = LogisticRegression(C=C, solver=solver, max_iter=max_iter, penalty=penalty)
    classifier.fit(X_train, y_train)

    predicted_stars = classifier.predict(X_test)
    accuracy = accuracy_score(y_test, predicted_stars)
    
    return accuracy

X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Maximize accuracy
study = optuna.create_study(direction='maximize')  
study.optimize(objective, n_trials=50)

best_params = study.best_params
print(f"Best hyperparameters: {best_params}")

best_classifier = LogisticRegression(**best_params)
best_classifier.fit(X_train, y_train)

#evaluation
final_predictions = best_classifier.predict(X_test)
final_accuracy = accuracy_score(y_test, final_predictions)
print(f"Final Accuracy: {final_accuracy}")


## Fast Text Embedding Model

In [ ]:
#TEXT MODEL FAST TEXT
from gensim.models import FastText


fasttext_model = FastText(sentences=train_reviews['tokens'], vector_size=50, window=5, min_count=5, epochs=10)
fasttext_model.save("fasttext_model.model")

In [ ]:
def get_review_vector(tokens, model):
    vectors = [model.wv[word] for word in tokens if word in model.wv]
    if len(vectors) == 0:
        return [0] * model.vector_size
    
    return sum(vectors) / len(vectors)

train_reviews['vector'] = train_reviews['tokens'].apply(lambda x: get_review_vector(x, fasttext_model))

X_train = list(train_reviews['vector'])
y_train = train_reviews['stars']

In [ ]:
# XGBOOST CLASSIFIER

from xgboost import XGBClassifier
y_train_adj = y_train - 1

classifier = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42)
classifier.fit(X_train, y_train_adj)

X_test = list(test_reviews['vector'])
predicted_stars = classifier.predict(X_test) + 1  


submission_df = pd.DataFrame({
    'review_id': test_reviews['review_id'],
    'stars': predicted_stars
})


# **Pruebas Datos No Textuales**

In [ ]:
# FEATURES SELECTION
usuarios=pd.read_csv('usuarios.csv', sep=',')

df = df.merge(usuarios, on='user_id',how='inner')

df2=df.drop(['review_id',	'user_id'	,'business_id','text', 'date', 'name_x','address','city','state','attributes','categories','name_y','elite','friends','hours','yelping_since','postal_code'],axis=1)
df2.rename(columns={'stars_x':'stars','name_y':'user_name','useful_x':'useful','funny_x':'funny','cool_x':'cool'}, inplace=True)
df2

In [ ]:
#NO TEXTUAL DATA NORMALIZATION

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
df2=df2[['funny','stars','cool','useful']]
label_encoder = LabelEncoder()


y_train = df2['stars'].round().astype(int)  
X_train = df2.drop(columns=['stars'])  


# Normalización
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

# Entrenamiento
classifier = LogisticRegression(max_iter=3000)
classifier.fit(X_train_scaled, y_train)

In [ ]:
# CLASSIFIER

#X_test_scaled = scaler.transform(test_reviews)
common_cols = list(set(X_train.columns) & set(test_reviews.columns))
X_test = test_reviews[common_cols]
X_test_scaled = scaler.fit_transform(X_test)

X_test_scaled
predicted_stars = classifier.predict(X_test_scaled)
predicted_stars

submission_df = pd.DataFrame({
    'review_id': test_reviews['review_id'],
    'stars': predicted_stars
})

submission_df.to_csv('submission.csv', index=False)

# **Optimización del modelo LightGBM**
**Preprocesamiento de los datos**
- Unión de las 3 tablas: reviews, users y bussinesses
- Adición de columnas como ```elite_years``` y ```categories```.

**Procesamiento de los textos de las reviews**
- TF-IDF con ```max_features=5000``` y 1-gramas
- Word2Vec con ```vector_size=100```.

**Modelo**
- Light GBM
- LightGBM optimizado con RandomizedSearch 
- LightGBM optimizado con Optuna

In [ ]:
# TF-IDF del texto
tfidf = TfidfVectorizer(max_features=5000)

X_text_train = tfidf.fit_transform(X_train[text_col])
X_text_test = tfidf.transform(X_test[text_col])

In [ ]:
# Modelo LightGBM
model = LGBMRegressor(n_estimators=100, learning_rate=0.1, max_depth=7)
model.fit(X_text_train, y_train)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 9.454706 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 869492
[LightGBM] [Info] Number of data points in the train set: 967784, number of used features: 5000
[LightGBM] [Info] Start training from score 3.759882


LGBMRegressor(max_depth=7)

Resultado inicial del MAE: `0.6922`

In [ ]:
from sklearn.impute import SimpleImputer
from scipy.sparse import hstack
from lightgbm import LGBMRegressor


cols_estructura = [
    'review_count_users', 'useful_users', 'funny_users', 'cool_users', 'fans',
    'average_stars', 'compliment_hot', 'compliment_more', 'compliment_profile',
    'compliment_cute', 'compliment_list', 'compliment_note', 'compliment_plain',
    'compliment_cool', 'compliment_funny', 'compliment_writer', 'compliment_photos',
    'stars_businesses', 'review_count_businesses', 'is_open',
    'useful_reviews', 'funny_reviews', 'cool_reviews',
    'review_month', 'review_weekday', 'text_len'
]

df_train['review_month'] = pd.to_datetime(df_train['date']).dt.month
df_train['review_weekday'] = pd.to_datetime(df_train['date']).dt.weekday

df_test['review_month'] = pd.to_datetime(df_test['date']).dt.month
df_test['review_weekday'] = pd.to_datetime(df_test['date']).dt.weekday

cols_estructura += ['review_month', 'review_weekday']

df_train['text_len'] = df_train['text'].apply(len)
df_test['text_len'] = df_test['text'].apply(len)
cols_estructura += ['text_len']

imputer = SimpleImputer(strategy='mean')

X_struct_train = imputer.fit_transform(df_train[cols_estructura])
X_struct_test = imputer.transform(df_test[cols_estructura])

X_train_final = hstack([X_text_train, X_struct_train])
X_test_final = hstack([X_text_test, X_struct_test])
y_train = df_train["stars_reviews"]

In [ ]:
model = LGBMRegressor(
    n_estimators=300,
    learning_rate=0.05,
    max_depth=10,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    n_jobs=-1
)

model.fit(X_train_final, y_train)


y_pred = model.predict(X_test_final)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 8.092455 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 873741
[LightGBM] [Info] Number of data points in the train set: 967784, number of used features: 5029
[LightGBM] [Info] Start training from score 3.759882


c:\Users\paula\anaconda3\envs\py311_env\Lib\site-packages\lightgbm\basic.py:1238: UserWarning: Converting data to scipy sparse matrix.
  _log_warning("Converting data to scipy sparse matrix.")


Al añadir estas columnas extra de información al modelo, el resultado baja a un `0.5385`

In [ ]:
# Vectorizar 'categories'
cat_vect = CountVectorizer(max_features=100, token_pattern='[^,]+')  # separa por coma
X_cat_train = cat_vect.fit_transform(df_train['categories'].fillna(''))
X_cat_test = cat_vect.transform(df_test['categories'].fillna(''))

In [ ]:
# Imputar valores faltantes en las columnas estructuradas
imputer = SimpleImputer(strategy='mean')
X_struct_train = imputer.fit_transform(df_train[cols_estructura])
X_struct_test = imputer.transform(df_test[cols_estructura])

# Concatenar texto + columnas
X_train_final = hstack([X_text_train, X_struct_train])
X_test_final = hstack([X_text_test, X_struct_test])
y_train = df_train["stars_reviews"]

In [ ]:
#Entrenar el modelo
model = LGBMRegressor(
    n_estimators=300,
    learning_rate=0.05,
    max_depth=10,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    n_jobs=-1
)

model.fit(X_train_final, y_train)

# Predicción
y_pred = model.predict(X_test_final)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 8.922104 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 873759
[LightGBM] [Info] Number of data points in the train set: 967784, number of used features: 5030
[LightGBM] [Info] Start training from score 3.759882


c:\Users\paula\anaconda3\envs\py311_env\Lib\site-packages\lightgbm\basic.py:1238: UserWarning: Converting data to scipy sparse matrix.
  _log_warning("Converting data to scipy sparse matrix.")


### RandomizedSearch

In [ ]:
# Modelo base
model = LGBMRegressor(random_state=42, n_jobs=-1)

param_dist = {
    'n_estimators': [100, 300, 500],
    'learning_rate': [0.01, 0.03, 0.05, 0.1],
    'max_depth': [7, 10, 12],
    'num_leaves': [31, 64, 128],
    'subsample': [0.7, 0.8, 1.0],
    'colsample_bytree': [0.7, 0.8, 1.0]
}

mae_scorer = make_scorer(mean_absolute_error, greater_is_better=False)

search = RandomizedSearchCV(
    estimator=model,
    param_distributions=param_dist,
    n_iter=40,
    scoring=mae_scorer,
    cv=3,
    verbose=2,
    random_state=42
)

start = time.time()

search.fit(X_train_final, y_train)

end = time.time()
elapsed_hours = round((end - start) / 3600, 2)

Fitting 3 folds for each of 40 candidates, totalling 120 fits
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 5.121050 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 872595
[LightGBM] [Info] Number of data points in the train set: 645189, number of used features: 5030
[LightGBM] [Info] Start training from score 3.761369
[CV] END colsample_bytree=1.0, learning_rate=0.03, max_depth=10, n_estimators=300, num_leaves=31, subsample=1.0; total time= 3.4min
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 6.796422 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 872972
[LightGBM] [Info] Number of data points in the train set: 645189, number of used features: 5030
[LightGBM] [Info] Start training from score 3.758269
[CV] END colsample_bytree=1.0, learning_rate=0.03, max_depth=10, n_estimators=300, num_leaves=31, subsample=1.0;

In [ ]:
best_model = search.best_estimator_
best_model.fit(X_train_final, y_train)

# Predicción
y_pred = best_model.predict(X_test_final)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 10.133734 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 873759
[LightGBM] [Info] Number of data points in the train set: 967784, number of used features: 5030
[LightGBM] [Info] Start training from score 3.759882


c:\Users\paula\anaconda3\envs\py311_env\Lib\site-packages\lightgbm\basic.py:1238: UserWarning: Converting data to scipy sparse matrix.
  _log_warning("Converting data to scipy sparse matrix.")


Obtenemos unos resultados de `0.4906`

### Randomized Search (versión 2)

In [ ]:
# Modelo base
model = LGBMRegressor(random_state=42, n_jobs=-1)

param_dist = {
    'n_estimators': [400, 500, 600],
    'learning_rate': [0.08, 0.1, 0.12],
    'max_depth': [10, 12, 14],
    'num_leaves': [48, 64, 80],
    'subsample': [0.7, 0.8, 0.9],
    'colsample_bytree': [0.9, 1.0]
}

mae_scorer = make_scorer(mean_absolute_error, greater_is_better=False)

search = RandomizedSearchCV(
    estimator=model,
    param_distributions=param_dist,
    n_iter=20,
    scoring=mae_scorer,
    cv=3,
    verbose=2,
    random_state=42
)

start = time.time()

search.fit(X_train_final, y_train)

end = time.time()
elapsed_hours = round((end - start) / 3600, 2)

Fitting 3 folds for each of 20 candidates, totalling 60 fits
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 6.876454 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 872595
[LightGBM] [Info] Number of data points in the train set: 645189, number of used features: 5030
[LightGBM] [Info] Start training from score 3.761369
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[CV] END colsample_bytree=1.0, learning_rate=0.12, max_depth=10, n_estimators=500, num_leaves=80, subsample=0.7; total time= 5.5min
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 9.599111 seconds.
You can set `force_row_wise=true` to remove the overhead

In [ ]:
best_model = search.best_estimator_
best_model.fit(X_train_final, y_train)
y_pred = best_model.predict(X_test_final)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 12.390065 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 873759
[LightGBM] [Info] Number of data points in the train set: 967784, number of used features: 5030
[LightGBM] [Info] Start training from score 3.759882


c:\Users\paula\anaconda3\envs\py311_env\Lib\site-packages\lightgbm\basic.py:1238: UserWarning: Converting data to scipy sparse matrix.
  _log_warning("Converting data to scipy sparse matrix.")


Resultado: `0.4837`

### Word2Vec

Probamos a procesar el texto con word2vec en vez de tfidf

In [ ]:
def tokenize(text):
    text = str(text).lower()
    text = re.sub(r"[^a-zA-Z\s]", "", text)  # quitar símbolos
    return nltk.word_tokenize(text)

df_train["tokens"] = df_train[text_col].fillna("").apply(tokenize)
df_test["tokens"] = df_test[text_col].fillna("").apply(tokenize)

In [ ]:
all_tokens = df_train["tokens"].tolist() + df_test["tokens"].tolist()
w2v_model = Word2Vec(sentences=all_tokens, vector_size=100, window=5, min_count=2, workers=4, sg=1, seed=42)

In [ ]:
# Promediar vectores por review

def get_review_vector(tokens, model, vector_size=100):
    vecs = [model.wv[word] for word in tokens if word in model.wv]
    if len(vecs) == 0:
        return np.zeros(vector_size)
    return np.mean(vecs, axis=0)

X_vec_train = np.vstack([get_review_vector(tokens, w2v_model, 100) for tokens in tqdm(df_train["tokens"])])
X_vec_test = np.vstack([get_review_vector(tokens, w2v_model, 100) for tokens in tqdm(df_test["tokens"])])

# Convertir a sparse
X_vec_train = csr_matrix(X_vec_train)
X_vec_test = csr_matrix(X_vec_test)

100%|██████████| 414765/414765 [02:20<00:00, 2960.24it/s]


In [ ]:
# Añadir longitud del texto como feature

df_train["text_len"] = df_train[text_col].fillna("").apply(len)
df_test["text_len"] = df_test[text_col].fillna("").apply(len)

X_struct_train = df_train[["text_len"]].values
X_struct_test = df_test[["text_len"]].values

imputer = SimpleImputer(strategy="mean")
X_struct_train = imputer.fit_transform(X_struct_train)
X_struct_test = imputer.transform(X_struct_test)

X_struct_train = csr_matrix(X_struct_train)
X_struct_test = csr_matrix(X_struct_test)

In [ ]:
# Añadir Word2Vec

X_train_final = hstack([X_vec_train, X_struct_train])
X_test_final = hstack([X_vec_test, X_struct_test])
y_train = df_train["stars_reviews"]

In [ ]:
# Entrenar LightGBM

model = LGBMRegressor(
    n_estimators=300,
    learning_rate=0.05,
    max_depth=10,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    n_jobs=-1
)
model.fit(X_train_final, y_train)

# Predicción
print("Generando predicciones...")
y_pred = model.predict(X_test_final)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.423514 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 25755
[LightGBM] [Info] Number of data points in the train set: 967784, number of used features: 101
[LightGBM] [Info] Start training from score 3.759882
Generando predicciones...


### Optuna

In [ ]:
# Dividir dataset
X_train_optuna, X_valid_optuna, y_train_optuna, y_valid_optuna = train_test_split(
    X_train_final, y_train, test_size=0.2, random_state=42
)

In [ ]:
def objective(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 100, 1000),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.2, log=True),
        "max_depth": trial.suggest_int("max_depth", 5, 15),
        "num_leaves": trial.suggest_int("num_leaves", 20, 128),
        "subsample": trial.suggest_float("subsample", 0.6, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.6, 1.0),
        "reg_alpha": trial.suggest_float("reg_alpha", 0.0, 10.0),
        "reg_lambda": trial.suggest_float("reg_lambda", 0.0, 10.0),
        "random_state": 42,
        "n_jobs": -1
    }

    model = LGBMRegressor(**params)
    model.fit(X_train_optuna, y_train_optuna)
    preds = model.predict(X_valid_optuna)
    mae = mean_absolute_error(y_valid_optuna, preds)
    return mae

In [ ]:
# Ejecutar optimización
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=100, show_progress_bar=True)

[I 2025-04-24 03:18:37,729] A new study created in memory with name: no-name-45dc03ed-d656-461a-9539-bf6650fa5b22


  0%|          | 0/100 [00:00<?, ?it/s]

[I 2025-04-24 03:24:12,542] Trial 0 finished with value: 0.4924679292489324 and parameters: {'n_estimators': 633, 'learning_rate': 0.197243382425006, 'max_depth': 9, 'num_leaves': 41, 'subsample': 0.9075642283992221, 'colsample_bytree': 0.7501553514091782, 'reg_alpha': 8.689173828201627, 'reg_lambda': 1.400309930031256}. Best is trial 0 with value: 0.4924679292489324.
[I 2025-04-24 03:27:06,761] Trial 1 finished with value: 0.5521995352025133 and parameters: {'n_estimators': 146, 'learning_rate': 0.04571695682647693, 'max_depth': 13, 'num_leaves': 55, 'subsample': 0.7259636219293324, 'colsample_bytree': 0.7514157319705483, 'reg_alpha': 2.8574906900584516, 'reg_lambda': 3.6079084332883617}. Best is trial 0 with value: 0.4924679292489324.
[I 2025-04-24 03:30:17,258] Trial 2 finished with value: 0.5013939566174803 and parameters: {'n_estimators': 193, 'learning_rate': 0.11217550903245539, 'max_depth': 12, 'num_leaves': 96, 'subsample': 0.8045275407693255, 'colsample_bytree': 0.96859533180

In [ ]:
best_model = LGBMRegressor(**study.best_params)
best_model.fit(X_train_final, y_train)

y_pred = best_model.predict(X_test_final)

c:\Users\paula\anaconda3\envs\py311_env\Lib\site-packages\lightgbm\basic.py:1238: UserWarning: Converting data to scipy sparse matrix.
  _log_warning("Converting data to scipy sparse matrix.")


Mejor resultado: ```0.4685```

# **Modelo TwoTower y ThreeTower**

El modelo Two-Tower (o modelo de dos torres) es una arquitectura de red neuronal que aprende representaciones (embeddings) separadas para diferentes tipos de entrada —en este caso, usuarios y textos de reseñas—, y luego las combina para predecir una puntuación.

**Arquitectura**

1. User Tower:

    - Recibe el user_id como entrada.

    - Aplica una capa de embedding para convertir el ID del usuario en un vector denso.

    - Pasa ese vector por una capa totalmente conectada (Linear) con activación ReLU.

    - Su objetivo es aprender una representación del usuario basada en su historial.

2. Review Tower:

    - Recibe la representación vectorial de la reseña (TF-IDF).

    - Procesa esta entrada mediante una capa lineal con ReLU.

    - Aprender una representación semántica del texto.

3. Head:

    - Concatena los vectores generados por ambas torres.

    - Pasa el vector combinado por una red neuronal profunda:

    - Capa Linear + Dropout + ReLU.

    - Capa final Linear que produce un único valor escalar (la predicción del rating).

In [ ]:
class YelpReviewDataset(Dataset):
    def __init__(self, df, df_users, text_vectorizer):
        self.df = df.reset_index(drop=True)
        self.df_users = df_users.set_index("user_id")
        self.vectorizer = text_vectorizer

        # Label Encoding de user_id
        self.user_encoder = LabelEncoder()
        self.df["user_id_encoded"] = self.user_encoder.fit_transform(self.df["user_id"])

        # Emparejamos info de usuarios
        #self.df["review_text_vec"] = list(self.vectorizer.transform(self.df["text"]).toarray())
        self.review_matrix = self.vectorizer.transform(self.df["text"])

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        user_id = row["user_id_encoded"]
        review_vec_sparse = self.review_matrix[idx]
        review_vec_dense = torch.tensor(review_vec_sparse.toarray(), dtype=torch.float32).squeeze(0)
        rating = torch.tensor(row["stars"], dtype=torch.float32)

        return user_id, review_vec_dense, rating




class YelpReviewTestDataset(Dataset):
    def __init__(self, df, df_users, text_vectorizer):
        self.df = df.reset_index(drop=True)
        self.df_users = df_users.set_index("user_id")
        self.vectorizer = text_vectorizer

        # Label Encoding de user_id
        self.user_encoder = LabelEncoder()
        self.df["user_id_encoded"] = self.user_encoder.fit_transform(self.df["user_id"])

        # Emparejamos info de usuarios
        #self.df["review_text_vec"] = list(self.vectorizer.transform(self.df["text"]).toarray())
        self.review_matrix = self.vectorizer.transform(self.df["text"])

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        user_id = row["user_id_encoded"]
        review_vec_sparse = self.review_matrix[idx]
        review_vec_dense = torch.tensor(review_vec_sparse.toarray(), dtype=torch.float32).squeeze(0)

        return user_id, review_vec_dense

In [ ]:
# Preentrena TF-IDF sobre el corpus completo
vectorizer = TfidfVectorizer(max_features=5000)
vectorizer.fit(df_train["text"])

In [ ]:
class UserTower(nn.Module):
    def __init__(self, num_users, embedding_dim=64, hidden_dim=128):
        super().__init__()
        self.user_embedding = nn.Embedding(num_users, embedding_dim)
        self.fc = nn.Sequential(
            nn.Linear(embedding_dim, hidden_dim),
            nn.ReLU()
        )

    def forward(self, user_id):
        x = self.user_embedding(user_id)
        return self.fc(x)

class ReviewTower(nn.Module):
    def __init__(self, input_dim=300, hidden_dim=128):
        super().__init__()
        self.fc = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU()
        )

    def forward(self, review_vec):
        return self.fc(review_vec)

class TwoTowerModel(nn.Module):
    def __init__(self, num_users, text_input_dim=300, hidden_dim=128):
        super().__init__()
        self.user_tower = UserTower(num_users, hidden_dim=hidden_dim)
        self.review_tower = ReviewTower(text_input_dim, hidden_dim=hidden_dim)
        self.head = nn.Sequential(
            nn.Linear(hidden_dim * 2, 256),
            nn.Dropout(0.2),
            nn.ReLU(),
            nn.Linear(256, 1)
        )

    def forward(self, user_id, review_vec):
        user_embed = self.user_tower(user_id)
        review_embed = self.review_tower(review_vec)
        combined = torch.cat([user_embed, review_embed], dim=-1)
        return self.head(combined).squeeze(1)

In [ ]:
import time

###################################
############## DATOS ##############
###################################
train_df, val_df = train_test_split(df_train, test_size=0.1, random_state=42)

train_dataset = YelpReviewDataset(train_df, df_users, vectorizer)
val_dataset = YelpReviewDataset(val_df, df_users, vectorizer)

train_loader = DataLoader(train_dataset, batch_size=256)
val_loader = DataLoader(val_dataset, batch_size=256)

In [ ]:
##########################################
############## MODELO ####################
##########################################
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = TwoTowerModel(num_users=len(train_dataset.user_encoder.classes_), text_input_dim=5000).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=3e-3)
criterion = nn.MSELoss()

print("--- 1/2 Entrenamiento ---")
for epoch in range(1):
    epoch_time = time.time()

    model.train()
    total_loss = 0
    for i, (user_ids, review_vecs, stars) in enumerate(train_loader):

        user_ids = user_ids.to(device)
        review_vecs = review_vecs.to(device)
        stars = stars.to(device)

        preds = model(user_ids, review_vecs)
        loss = criterion(preds, stars)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    epoch_end_time = time.time()
    epoch_time = epoch_end_time - epoch_time
    print(f"Epoch {epoch+1}, Loss: {total_loss / len(train_loader):.4f}, Time: {epoch_time:.2f}s")

print("")
print("--- Validación ---")
model.eval()
total_loss = 0
with torch.no_grad():
    for i, (user_ids, review_vecs, stars) in enumerate(val_loader):
        user_ids = user_ids.to(device)
        review_vecs = review_vecs.to(device)
        stars = stars.to(device)

        preds = model(user_ids, review_vecs)
        loss = criterion(preds, stars)
        total_loss += loss.item()

print(f"Validation Loss: {total_loss / len(val_loader):.4f}")

> Continuación del entrenamiento con los datos de validación

In [ ]:
print("--- 2/2 Entrenamiento ---")
for epoch in range(1):
    epoch_time = time.time()

    model.train()
    total_loss = 0
    for i, (user_ids, review_vecs, stars) in enumerate(val_loader):

        user_ids = user_ids.to(device)
        review_vecs = review_vecs.to(device)
        stars = stars.to(device)

        preds = model(user_ids, review_vecs)
        loss = criterion(preds, stars)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    epoch_end_time = time.time()
    epoch_time = epoch_end_time - epoch_time
    print(f"Epoch {epoch+1}, Loss: {total_loss / len(val_loader):.4f}, Time: {epoch_time:.2f}s")

In [ ]:
import gc
import torch

# # Elimina referencias explícitamente
# del train_df, val_df, train_dataset, val_dataset, train_loader, val_loader

# # Limpia el recolector de basura de Python
# gc.collect()


# review_id, stars
test_dataset = YelpReviewTestDataset(df_test, df_users, vectorizer)
test_loader = DataLoader(test_dataset, batch_size=256)

# Test set
model.eval()
predictions = []
with torch.no_grad():
    for i, (user_ids, review_vecs) in enumerate(test_loader):
        user_ids = user_ids.to(device)
        review_vecs = review_vecs.to(device)
        preds = model(user_ids, review_vecs)
        predictions.extend(preds.cpu().numpy())
        if i % 100 == 0:
            print(f"Batch {i}/{len(test_loader)}")


> Prueba con TF-IDF de 20000

In [ ]:
##############
### TF-IDF ###
##############
vectorizer = TfidfVectorizer(max_features=20000)
vectorizer.fit(df_train["text"])

import time

###################################
############## DATOS ##############
###################################
train_df, val_df = train_test_split(df_train, test_size=0.1, random_state=42)

train_dataset = YelpReviewDataset(train_df, df_users, vectorizer)
val_dataset = YelpReviewDataset(val_df, df_users, vectorizer)

train_loader = DataLoader(train_dataset, batch_size=256)
val_loader = DataLoader(val_dataset, batch_size=256)

#######################################################
############## ENTRENAMIENTO Y VALIDACIÓN #############
#######################################################

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = TwoTowerModel(num_users=len(train_dataset.user_encoder.classes_), text_input_dim=20000).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=4e-3)
criterion = nn.MSELoss()

print("--- 1/2 Entrenamiento ---")
for epoch in range(1):
    epoch_time = time.time()

    model.train()
    total_loss = 0
    for i, (user_ids, review_vecs, stars) in enumerate(train_loader):

        user_ids = user_ids.to(device)
        review_vecs = review_vecs.to(device)
        stars = stars.to(device)

        preds = model(user_ids, review_vecs)
        loss = criterion(preds, stars)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    epoch_end_time = time.time()
    epoch_time = epoch_end_time - epoch_time
    print(f"Epoch {epoch+1}, Loss: {total_loss / len(train_loader):.4f}, Time: {epoch_time:.2f}s")

print("")
print("--- Validación ---")
model.eval()
total_loss = 0
with torch.no_grad():
    for i, (user_ids, review_vecs, stars) in enumerate(val_loader):
        user_ids = user_ids.to(device)
        review_vecs = review_vecs.to(device)
        stars = stars.to(device)

        preds = model(user_ids, review_vecs)
        loss = criterion(preds, stars)
        total_loss += loss.item()

print(f"Validation Loss: {total_loss / len(val_loader):.4f}")


print("--- 2/2 Entrenamiento ---")
for epoch in range(1):
    epoch_time = time.time()

    model.train()
    total_loss = 0
    for i, (user_ids, review_vecs, stars) in enumerate(val_loader):

        user_ids = user_ids.to(device)
        review_vecs = review_vecs.to(device)
        stars = stars.to(device)

        preds = model(user_ids, review_vecs)
        loss = criterion(preds, stars)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    epoch_end_time = time.time()
    epoch_time = epoch_end_time - epoch_time
    print(f"Epoch {epoch+1}, Loss: {total_loss / len(val_loader):.4f}, Time: {epoch_time:.2f}s")


##########################################
############## PREDICCIONES ##############
##########################################

test_dataset = YelpReviewTestDataset(df_test, df_users, vectorizer)
test_loader = DataLoader(test_dataset, batch_size=256)

model.eval()
predictions = []
with torch.no_grad():
    for i, (user_ids, review_vecs) in enumerate(test_loader):
        user_ids = user_ids.to(device)
        review_vecs = review_vecs.to(device)
        preds = model(user_ids, review_vecs)
        predictions.extend(preds.cpu().numpy())
        if i % 100 == 0:
            print(f"Batch {i}/{len(test_loader)}")

submission_df = pd.DataFrame({
    'review_id': df_test['review_id'],
    'stars': predictions
})
submission_df.to_csv('prediction_TwoTowers_8.csv', index=False)

### Segundo Experimento. TF-IDF + ThreeTower

In [ ]:
class YelpReviewDataset(Dataset):
    def __init__(self, df, df_users, df_businesses, text_vectorizer):
        self.df = df.reset_index(drop=True)
        self.df_users = df_users.set_index("user_id")
        self.df_businesses = df_businesses.set_index("business_id")
        self.vectorizer = text_vectorizer

        # Label Encoding de user_id y business_id
        self.user_encoder = LabelEncoder()
        self.business_encoder = LabelEncoder()

        self.df["user_id_encoded"] = self.user_encoder.fit_transform(self.df["user_id"])
        self.df["business_id_encoded"] = self.business_encoder.fit_transform(self.df["business_id"])

        # Vectorizamos el texto
        self.review_matrix = self.vectorizer.transform(self.df["text"])

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]

        user_id = row["user_id_encoded"]
        business_id = row["business_id_encoded"]

        review_vec_sparse = self.review_matrix[idx]
        review_vec_dense = torch.tensor(review_vec_sparse.toarray(), dtype=torch.float32).squeeze(0)

        rating = torch.tensor(row["stars"], dtype=torch.float32)

        return user_id, business_id, review_vec_dense, rating


class YelpReviewTestDataset(Dataset):
    def __init__(self, df, df_users, df_businesses, text_vectorizer):
        self.df = df.reset_index(drop=True)
        self.df_users = df_users.set_index("user_id")
        self.df_businesses = df_businesses.set_index("business_id")
        self.vectorizer = text_vectorizer

        # Label Encoding de user_id y business_id
        self.user_encoder = LabelEncoder()
        self.business_encoder = LabelEncoder()

        self.df["user_id_encoded"] = self.user_encoder.fit_transform(self.df["user_id"])
        self.df["business_id_encoded"] = self.business_encoder.fit_transform(self.df["business_id"])

        # Vectorizamos el texto
        self.review_matrix = self.vectorizer.transform(self.df["text"])

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]

        user_id = row["user_id_encoded"]
        business_id = row["business_id_encoded"]

        review_vec_sparse = self.review_matrix[idx]
        review_vec_dense = torch.tensor(review_vec_sparse.toarray(), dtype=torch.float32).squeeze(0)

        # En el dataset de test no necesitamos el rating, por eso no lo devolvemos
        return user_id, business_id, review_vec_dense


In [ ]:
# Preentrena TF-IDF sobre el corpus completo
vectorizer = TfidfVectorizer(max_features=5000)
vectorizer.fit(df_train["text"])

In [ ]:
class UserTower(nn.Module):
    def __init__(self, num_users, embedding_dim=64, hidden_dim=128):
        super().__init__()
        self.user_embedding = nn.Embedding(num_users, embedding_dim)
        self.fc = nn.Sequential(
            nn.Linear(embedding_dim, hidden_dim),
            nn.ReLU()
        )

    def forward(self, user_id):
        x = self.user_embedding(user_id)
        return self.fc(x)

class BusinessTower(nn.Module):
    def __init__(self, num_businesses, embedding_dim=64, hidden_dim=128):
        super().__init__()
        self.business_embedding = nn.Embedding(num_businesses, embedding_dim)
        self.fc = nn.Sequential(
            nn.Linear(embedding_dim, hidden_dim),
            nn.ReLU()
        )

    def forward(self, business_id):
        x = self.business_embedding(business_id)
        return self.fc(x)

class ReviewTower(nn.Module):
    def __init__(self, input_dim=300, hidden_dim=128):
        super().__init__()
        self.fc = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU()
        )

    def forward(self, review_vec):
        return self.fc(review_vec)

class ThreeTowerModel(nn.Module):
    def __init__(self, num_users, num_businesses, text_input_dim=300, hidden_dim=128):
        super().__init__()
        # Aseguramos que las dimensiones sean correctas para cada torre
        self.user_tower = UserTower(num_users, embedding_dim=64, hidden_dim=hidden_dim)
        self.business_tower = BusinessTower(num_businesses, embedding_dim=64, hidden_dim=hidden_dim)
        self.review_tower = ReviewTower(input_dim=text_input_dim, hidden_dim=hidden_dim)

        self.head = nn.Sequential(
            nn.Linear(hidden_dim * 3, 256),  # Multiplicamos por 3, ya que concatenamos user, business y review
            nn.Dropout(0.2),
            nn.ReLU(),
            nn.Linear(256, 1)
        )

    def forward(self, user_id, business_id, review_vec):
        # Hacemos que las tres torres devuelvan embeddings compatibles
        user_embed = self.user_tower(user_id)
        business_embed = self.business_tower(business_id)
        review_embed = self.review_tower(review_vec)

        # Concatenamos las representaciones de las tres torres
        combined = torch.cat([user_embed, business_embed, review_embed], dim=-1)

        # Pasa la concatenación por la cabeza para hacer la predicción final
        return self.head(combined).squeeze(1)


In [ ]:
###################################
############## DATOS ##############
###################################

# Split de entrenamiento/validación
train_df, val_df = train_test_split(df_train, test_size=0.1, random_state=42)

train_dataset = YelpReviewDataset(train_df, df_users, df_businesses, vectorizer)
val_dataset = YelpReviewDataset(val_df, df_users, df_businesses, vectorizer)

train_loader = DataLoader(train_dataset, batch_size=256)
val_loader = DataLoader(val_dataset, batch_size=256)

#######################################################
############## ENTRENAMIENTO Y VALIDACIÓN #############
#######################################################

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [ ]:
# Ajusta el modelo para que reciba tanto el user_id como el business_id
model = ThreeTowerModel(
    num_users=len(train_dataset.user_encoder.classes_),  # Número de usuarios
    num_businesses=len(train_dataset.business_encoder.classes_),  # Número de negocios (tienes que definir un encoder para 'business_id')
    text_input_dim=5000  # Ajusta según la dimensión de tu entrada de texto
).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=4e-3)
criterion = nn.MSELoss()

print("--- 1/2 Entrenamiento ---")
for epoch in range(1):
    epoch_time = time.time()

    model.train()
    total_loss = 0
    for i, (user_ids, business_ids, review_vecs, stars) in enumerate(train_loader):

        user_ids = user_ids.to(device)
        business_ids = business_ids.to(device)  # Ahora también tomamos business_id
        review_vecs = review_vecs.to(device)
        stars = stars.to(device)

        preds = model(user_ids, business_ids, review_vecs)
        loss = criterion(preds, stars)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    epoch_end_time = time.time()
    epoch_time = epoch_end_time - epoch_time
    print(f"Epoch {epoch+1}, Loss: {total_loss / len(train_loader):.4f}, Time: {epoch_time:.2f}s")

print("")
print("--- Validación ---")
model.eval()
total_loss = 0
with torch.no_grad():
    for i, (user_ids, business_ids, review_vecs, stars) in enumerate(val_loader):
        user_ids = user_ids.to(device)
        business_ids = business_ids.to(device)  # Ahora también tomamos business_id
        review_vecs = review_vecs.to(device)
        stars = stars.to(device)

        preds = model(user_ids, business_ids, review_vecs)
        loss = criterion(preds, stars)
        total_loss += loss.item()

print(f"Validation Loss: {total_loss / len(val_loader):.4f}")

print("--- 2/2 Entrenamiento ---")
for epoch in range(1):
    epoch_time = time.time()

    model.train()
    total_loss = 0
    for i, (user_ids, business_ids, review_vecs, stars) in enumerate(val_loader):

        user_ids = user_ids.to(device)
        business_ids = business_ids.to(device)  # Ahora también tomamos business_id
        review_vecs = review_vecs.to(device)
        stars = stars.to(device)

        preds = model(user_ids, business_ids, review_vecs)
        loss = criterion(preds, stars)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    epoch_end_time = time.time()
    epoch_time = epoch_end_time - epoch_time
    print(f"Epoch {epoch+1}, Loss: {total_loss / len(val_loader):.4f}, Time: {epoch_time:.2f}s")

##########################################
############## PREDICCIONES ##############
##########################################

# Dataset de test con business_id también
test_dataset = YelpReviewTestDataset(df_test, df_users, df_businesses, vectorizer)
test_loader = DataLoader(test_dataset, batch_size=256)

model.eval()
predictions = []
with torch.no_grad():
    for i, (user_ids, business_ids, review_vecs) in enumerate(test_loader):
        user_ids = user_ids.to(device)
        business_ids = business_ids.to(device)  # Ahora también tomamos business_id
        review_vecs = review_vecs.to(device)
        preds = model(user_ids, business_ids, review_vecs)
        predictions.extend(preds.cpu().numpy())
        if i % 100 == 0:
            print(f"Batch {i}/{len(test_loader)}")

# Guardar las predicciones en el archivo de salida
submission_df = pd.DataFrame({
    'review_id': df_test['review_id'],
    'stars': predictions
})
submission_df.to_csv('prediction_TwoTowers_with_business.csv', index=False)
